# DS4024 Automatic query extraction

I'm getting tired of manually implementing the API to this device; let's try doing a bit of codegen.  This is a very janky bit of groundwork that gets turned into real code in the actual library.

Before you implement any new commands: **KNOW WHAT YOU ARE DOING**.  Refer closely to the manual before running any command, as there are a few which are possibly "destructive."  For instance, there is the ability to uninstall installed software options: reinstalling these via the scope's text input mechanism is Not Fun.  There is also a factory reset option, which may or may not require re-running self cal afterwards.  Everything I know of is recoverable, but a huge pain in the butt to recover from.


## Processing


First up, we run the PDF through `pdftotext` to get a text file, and now processing it a bit:

```
# Rename the file to get rid of ampersands.
0] jbm@complexity:~/Desktop $ mv MSO\&DS4000_programming.pdf DS4000_programming.pdf 
0] jbm@complexity:~/Desktop $ pdftotext DS4000_programming.pdf 
```

We'll then parse this down into a tree structure for all the query (`:FOO:BAR?`) commands.  Note that this does not capture whether or not they take arguments!  That's something you will need to do for yourself, cross-referencing the programming guide (Rigol Publication Number: PGA21101-1110).

However, this gets you a copy of the complete set of query commands to bum down and/or tweak, which saves a lot of pointless typing.  For an example of what motivates this, see DS4024_SCPI's `_fetch_trigger_settings()` method: by doing it this way, we can collect all the settings for all the trigger modes in a single place, without writing huge swathes of code.



In [1]:
f = open("/home/jbm/Desktop/DS4000_programming.txt")
lines = f.readlines()
f.close()

len(lines)

17422

In [2]:
import re

In [3]:
def build_query_tree(family):
    query_re = re.compile(f"^:{family}.*\\?")
    
    rel_lines = [l for l in lines if query_re.match(l)]
    
    
    RE_LOWERCASE = re.compile('[a-z]')  # Yes UTF yes PCREs yes yes yes, but: ease of understanding
    def _parse_cmd(l):
        l = l.strip()
        toplevel_toks = l.split()
        cmd = toplevel_toks[0]
        args = toplevel_toks[1:]
        cmd_uc = RE_LOWERCASE.subn('', cmd)[0]
        toks = cmd_uc.split(":")
        if toks[0] == '':
            toks = toks[1:]
        return (toks, args)

    result = {'': set()}
    for l in rel_lines:
        if "......" in l:
            # Index entries
            continue
            
        cmd,parms = _parse_cmd(l)
        
        if len(cmd) == 2:
            result[""].add(cmd[1][:-1])
            continue
        if len(cmd) == 3:
            _,subfam,tgt = cmd
            if subfam not in result:
                result[subfam] = set()
            result[subfam].add(tgt[:-1])

    return result

In [4]:
build_query_tree("CALC")

{'': {'MODE'},
 'ADV': {'EXPR', 'INV', 'VAR1', 'VAR2', 'VOFF', 'VSC'},
 'ADD': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'DIV': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'FFT': {'HCEN',
  'HOFF',
  'HSC',
  'HSP',
  'SOUR',
  'SPL',
  'VOFF',
  'VSC',
  'VSM',
  'WIND'},
 'LOG': {'ATHR', 'BTHR', 'INV', 'OPER', 'SA', 'SB', 'VOFF', 'VSC'},
 'MULT': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'SUB': {'INV', 'SA', 'SB', 'VOFF', 'VSC'}}

Okay, that looks good, now let's parse the whole freaking thing into a single dictionary:

In [5]:
def build_full_query_tree():
    query_re = re.compile(f"^:.*\\?")
    
    rel_lines = [l for l in lines if query_re.match(l)]
    
    
    RE_LOWERCASE = re.compile('[a-z]')  # Yes UTF yes PCREs yes yes yes, but: ease of understanding
    def _parse_cmd(l):
        l = l.strip()
        toplevel_toks = l.split()
        cmd = toplevel_toks[0]
        args = toplevel_toks[1:]
        cmd_uc = RE_LOWERCASE.subn('', cmd)[0]
        toks = cmd_uc.split(":")
        if toks[0] == '':
            toks = toks[1:]
        return (toks, args)

    result = {'': set()}
    for l in rel_lines:
        if "......" in l:
            # Index entries
            continue
            
        cmd,parms = _parse_cmd(l)
        
        family = cmd[0]
        if family not in result:
            result[family] = { "": set() }
        
        if len(cmd) == 2:
            family,tgt = cmd
            result[family][""].add(tgt[:-1])
            continue
        if len(cmd) == 3:
            family,subfam,tgt = cmd
            if subfam not in result[family]:
                result[family][subfam] = set()
            result[family][subfam].add(tgt[:-1])

    return result


fqt = build_full_query_tree()

print("Complete list of query commands:")

for family in fqt: 
    
    if family == "":    # Dump leaf nodes
        for tgt in fqt[family]:
            print(f':{tgt}?')
        continue
        
    print(f'=== {family} ======')
    for subfam in fqt[family]:
        if subfam == "":
            for tgt in fqt[family][subfam]:
                print(f'     :{tgt}?')
            continue
        print(f'  {subfam:}')
        for tgt in fqt[family][subfam]:
            print(f'     ::{tgt}?')

Complete list of query commands:
=== CALC ======
     :MODE?
  ADV
     ::VSC?
     ::VAR2?
     ::EXPR?
     ::INV?
     ::VAR1?
     ::VOFF?
  ADD
     ::VSC?
     ::INV?
     ::SA?
     ::SB?
     ::VOFF?
  DIV
     ::VSC?
     ::INV?
     ::SA?
     ::SB?
     ::VOFF?
  FFT
     ::HOFF?
     ::SOUR?
     ::VSC?
     ::WIND?
     ::HCEN?
     ::HSP?
     ::HSC?
     ::SPL?
     ::VSM?
     ::VOFF?
  LOG
     ::VSC?
     ::BTHR?
     ::OPER?
     ::INV?
     ::SA?
     ::SB?
     ::ATHR?
     ::VOFF?
  MULT
     ::VSC?
     ::INV?
     ::SA?
     ::SB?
     ::VOFF?
  SUB
     ::VSC?
     ::INV?
     ::SA?
     ::SB?
     ::VOFF?
=== MEAS ======
     :FPH?
     :MPAR?
     :PER?
     :VPP?
     :PWID?
     :AREA?
     :FDEL?
     :MAR?
     :R2FD?
     :VMAX?
     :PVRM?
     :OVER?
     :VMIN?
     :VAVG?
     :RPH?
     :F2RP?
     :VAMP?
     :VRMS?
     :RDEL?
     :RTIM?
     :AMS?
     :NWID?
     :VBAS?
     :NDUT?
     :FREQ?
     :SOUR?
     :VTOP?
     :PDUT?
     :FTIM?
   

That smells right.  Let's check a few of the dictionaries that are actually in use in the code:

In [6]:
fqt["CALC"]

{'': {'MODE'},
 'ADV': {'EXPR', 'INV', 'VAR1', 'VAR2', 'VOFF', 'VSC'},
 'ADD': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'DIV': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'FFT': {'HCEN',
  'HOFF',
  'HSC',
  'HSP',
  'SOUR',
  'SPL',
  'VOFF',
  'VSC',
  'VSM',
  'WIND'},
 'LOG': {'ATHR', 'BTHR', 'INV', 'OPER', 'SA', 'SB', 'VOFF', 'VSC'},
 'MULT': {'INV', 'SA', 'SB', 'VOFF', 'VSC'},
 'SUB': {'INV', 'SA', 'SB', 'VOFF', 'VSC'}}

In [7]:
fqt["WAV"]

{'': {'DATA',
  'FORM',
  'MODE',
  'POIN',
  'PRE',
  'SOUR',
  'STAR',
  'STAT',
  'STOP',
  'XINC',
  'XOR',
  'XREF',
  'YINC',
  'YOR',
  'YREF'}}

This looks pretty good.  I'm not going to bother cross-checking everything right now, as I will probably never use 90% of it.  If a complete implementation is intended, it seems prudent to wrap all of the above up in per-family classes, to minimize jank.